In [6]:
from imutils import contours
from skimage import measure
import numpy as np
import argparse
import imutils
import cv2
import matplotlib.pyplot as plt
import os
import pandas as pd
from PIL import Image, ImageOps

In [7]:
# set path to folder with galaxy images
path = '..\\Data\\photometric_jpeg_09.12.21\\'

# make directory for resized images
try:
    os.makedirs('resized')
except:
    print("Folder for resized images already exists")
    
# choose all images in the folder
optical_jpegs = np.array(os.listdir(path))

In [8]:
def resize_with_padding(img, expected_size):
    img.thumbnail((expected_size[0], expected_size[1]))
    # print(img.size)
    delta_width = expected_size[0] - img.size[0]
    delta_height = expected_size[1] - img.size[1]
    pad_width = delta_width // 2
    pad_height = delta_height // 2
    padding = (pad_width, pad_height, delta_width - pad_width, delta_height - pad_height)
    return ImageOps.expand(img, padding, fill='black')

In [ ]:
for pic in optical_jpegs:
    image = cv2.imread(path + pic)
    gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
    blurred = cv2.GaussianBlur(gray, (11, 11), 0)
    
    # threshold the image to reveal light regions in the blurred image
    thresh = cv2.threshold(blurred, 40, 255, cv2.THRESH_BINARY)[1]
    
    # perform a series of erosions and dilations to remove
    # any small blobs of noise from the thresholded image
    thresh = cv2.erode(thresh, None, iterations=2)
    thresh = cv2.dilate(thresh, None, iterations=4)
    
    # perform a connected component analysis on the thresholded
    # image, then initialize a mask to store only the "large"
    # components
    labels = measure.label(thresh, connectivity=2, background=0)
    mask = np.zeros(thresh.shape, dtype="uint8")
    
    # loop over the unique components
    for label in np.unique(labels):
        # if this is the background label which is 0, ignore it
        if label == 0:
            continue
        # otherwise, construct the label mask and count the number of pixels 
        labelMask = np.zeros(thresh.shape, dtype="uint8")
        labelMask[labels == label] = 255
        numPixels = cv2.countNonZero(labelMask)

        # if the number of pixels in the component is sufficiently
        # large, then add it to our mask of "large blobs"
        if numPixels > 200:
            mask = cv2.add(mask, labelMask)
            
        # find the contours in the mask
        cnts = cv2.findContours(mask.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        cnts = imutils.grab_contours(cnts)
        
        # define the center pixel
        center_pixel = (image.shape[0]/2, image.shape[1]/2)

        # compute all blobs and choose the one that is closest to center
        for (i, c) in enumerate(cnts):
            # compute the boundig box for all blobs
            (x, y, w, h) = cv2.boundingRect(c)
            # cover the object with minimum area
            ((cX, cY), radius) = cv2.minEnclosingCircle(c)
            
            # choose only the object to which the center pixel belongs
            if(x < center_pixel[0] and center_pixel[0] < (x+w) and y < center_pixel[1] and center_pixel[1] < (y+h)):
                to_crop = c
        
        """
        Optional TODO: Add function, so that only the minimum circle area gets cut out and everything else is black
        """
        
        try:
            (x, y, w, h) = cv2.boundingRect(to_crop)
            ((cX, cY), radius) = cv2.minEnclosingCircle(c)

            # Cropping the image with a tolerance
            cropped = image[y-20:y+h+20, x-20:x+w+20]

            img = Image.fromarray(cropped, 'RGB')
            resized = resize_with_padding(img, (300, 300))
            plt.axis('off')
            plt.imshow(resized)
            plt.savefig(path + 'resized\\' + pic, dpi=300, pad_inches=0, bbox_inches='tight', transparent=True)
            
        except:
            print("There was an error at picture", pic)